In [ ]:
#Mouting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Libraries

In [ ]:
#Install Pyradiomics
!pip install pyradiomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import skimage
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import gc
from tqdm import tqdm
import copy
import pandas as pd

#Local binary pattern
from skimage.feature import local_binary_pattern

#GLCM
from radiomics.glcm import RadiomicsGLCM
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import SimpleITK as sitk
import six
import pickle

In [ ]:
#Global dictionaries
mag_dict = {0:'40',1:'100',2:'200',3:'400'}
tt_dict = {0:'train',1:'test'}

# Functions

## LBP

In [ ]:
def get_LBP(path_folder):
  """
  Compute the rotation invariant uniform (frequent) LBP feature from all images in a folder.
  Returns the feature matrix mxn (m patient, n features)
  :param path_folder: list of paths of all images in given folder
  :return: feature matrix mxn
  """
  lbp_feature_vector=[]
  for path in tqdm(path_folder): 
      im = cv.cvtColor(cv.imread(path),cv.COLOR_BGR2GRAY);
      im_lbp = local_binary_pattern(im,P=8,R=1,method='uniform')
      hist = np.histogram(im_lbp.ravel())
      feature_array=np.ndarray.tolist(hist[0])
      #Add endpoint. Benign=0, Malignant=1.
      feature_array.append(int(path.find('_M_')!=-1))
      #Add patient ID
      feature_array.insert(0,path.rsplit('/', 1)[1].split('-', 3)[2])
      #Save feature array in list
      lbp_feature_vector.append(feature_array)
  return lbp_feature_vector

## GLCM

In [ ]:
#### GLCM
####
def extractor_setting(feature_names):
  """
  Settings of the feature extraction
  
  :param feature_names: list of the names of the features to be extracted
      e.g. features_names = ['JointEnergy', 'Contrast','Correlation']
  :return: extractor with pre-defined settings and the # of features (length of feature_names)
  """

  settings = {}
  settings['binWidth'] = 1 #To ensure a 1 pixel width for the histogram onece the image has been requantized to 8 gray levels.
  settings['correctMask'] = True #To ensure the mask and the image are in the same coordinates
  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  #extractor.enableFeatureClassByName('glcm') #Enable all GLCM features
  extractor.enableFeaturesByName(glcm=feature_names)
  num_features = len(feature_names)
  print('Extraction parameters:\n\t', extractor.settings)
  print('Enabled filters:\n\t', extractor.enabledImagetypes)
  print('Enabled features:\n\t', extractor.enabledFeatures)
  return extractor, num_features

def GLCM_feature_im(img_path, extractor, num_features):
  """
  Description: Extract GLCM features from an image. Requantization to 8 gray levels.

  :param img_path: Path of image from which the features willbe obtain
  :param extractor: Pyradiomics extractor defined in function extractor_settings
  :param num_features: # of features given by function extractor_settings.
  :return: feature vector
  """

  #Read image
  img = cv.cvtColor(cv.imread(img_path),cv.COLOR_BGR2GRAY)#Read and to grayscale
  im = np.uint8(np.digitize(img, np.arange(0,256, 32))) - 1 #Normalize to 8 gray level
  im_im = sitk.GetImageFromArray(im) #Pass to sitk objet
  #mask
  mask = np.ones(im.shape).astype(np.uint8) #Create maskwith same shape as image
  mask[-1,-1]=0 #Hack to allow full image segmentation. Last value is set to 0 so it wont be counted in the GLCM computation
  mask_im = sitk.GetImageFromArray(mask) #Pass to sitk object
  #Extract
  result = extractor.execute(im_im,mask_im) # Extract features
  features = np.array(list(result.values())[-num_features:]) #save as array
  return features

def GLCM_for_folder(paths, f, tt, mag, num_features, extractor):
  """
  Definition: Extract feature vector for all images in a folder, saved as a pickle numpy matrix

  :param paths: paths of images
  :param f: fold number-1
  :param tt: train or test encoded
  :param mag: magnification encoded using dictionary
  :param num_features: Number of features ot be extracted. It defines the size of the feature matrix
  :param extractor: Pyradiomics extractor with presettings
  :return: NONE (Pickle file will be saved in drive)
  """

  paths_folder = paths[f,tt,mag] #Paths of the first folder (There are 40)
  n = len(paths_folder) #number of images (samples)
  feature_matrix = np.zeros((n,num_features),dtype=np.float32) #Shape of feature matrix set

  #Extraction of features
  for i in tqdm(range(n)):
    img_path = paths_folder[i] #Image from which features are extracted
    ft_v = GLCM_feature_im(img_path, extractor, num_features) #Returns feature vector of m-dimension
    feature_matrix[i,:] = ft_v
  #Saving document
  file_name = f'/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/fold{f+1}/{tt_dict[tt]}/GLCM_f{f+1}_{tt_dict[tt]}_{mag_dict[mag]}x_fv.p'
  with open(file_name, 'wb') as handle:
      pickle.dump(feature_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

## PFTAS

In [ ]:
def PFTAS(image_path):
  #This function gives PFTAS per !IMAGE!
  chs = cv.imread(image_path)
  feature_vector = np.array([]) #Array with final 81 features
  for i in range(3): #Three channels
    ch = chs[:,:,i] #Obtain one channel image
    ch_T,_ = cv.threshold(ch,0,255,type=cv.THRESH_BINARY + cv.THRESH_OTSU) #Otsu T
    mu = ((ch>ch_T)*ch).sum()/np.sum(ch>ch_T) #Mean value for pixels above Otsu T
    std_ch = ch[ch>ch_T].std() #Std of pixels above threshold
    
    #Define windows for binarization
    w1 = ((ch > (mu - std_ch))*(ch < (mu + std_ch)));
    w2 = (ch > (mu - std_ch))
    w3 = (ch > mu)
    window = {0: w1, 1: w2, 2: w3, 3: ~w1, 4: ~w2, 5: ~w3} #Put all thresholding criteria together
    
    feature_vector_ch = np.array([]) #(This will contain the 27 features of the channel)*2 because of bitwise inversion
    for w in window:
      fv_w = feature_v_per_window(window[w])
      feature_vector_ch = np.append(feature_vector_ch,fv_w) #Append the (27 features)*2
    feature_vector = np.append(feature_vector,feature_vector_ch) #Append the (81 vectors)*2
  return feature_vector

def feature_v_per_window(window):
  #Function used inside PFTAS. It computes the number of white neighbor in each window
  thresh = window.astype(np.uint8)
  ker = np.ones((3, 3))
  ker[1,1]=10; #The value 10 is given to separate black and white central pixels
  ch_conv = cv.filter2D(thresh,kernel=ker,ddepth=-1,borderType=cv.BORDER_REFLECT) #Convolution to count neighbors
  values = np.histogram(ch_conv,bins=range(20))[0][-9:] #Last 9 values are the white pixels histgram values
  feature_v = values/float(values.sum())
  return feature_v

def get_PFTAS(path_folder):
  #This function give PFTAS list per !FOLDER!
  PFTAS_feature_vector=[]
  for path in tqdm(path_folder):
      feature_array=np.ndarray.tolist(PFTAS(path))
      #Add endpoint. Benign=0, Malignant=1.
      feature_array.append(int(path.find('_M_')!=-1))
      #Add patient ID
      feature_array.insert(0,path.rsplit('/', 1)[1].split('-', 3)[2])
      #Save feature array in list
      PFTAS_feature_vector.append(feature_array)
  return PFTAS_feature_vector

## Gabor features

In [ ]:
## Creation of Gabor Filters
def Gabor(path):
   # Gabor filters are band pass filters, they allow certain frequencies and reject others.
   #read the image for each path. 
   im = cv.cvtColor(cv.imread(path), cv.COLOR_BGR2GRAY)
   ## reshape the image and put it in our feature vector as a first entity
   img2 = im.reshape(-1)
   df = pd.DataFrame()
   df['1'] = img2
   #Generate Gabor features
   num = 2  #To count numbers up in order to give Gabor features a label in the data frame
   kernels = []  #Create empty list to hold all kernels that we will generate in a loop
   lamda = np.pi / 4
   psi = 0
   for theta in range(2):   #Define number of thetas. Here 4 theta values 0 and 1/4 . pi 1/2pi and 3pi/2 (all 4 directions)
       theta = theta / 4. * np.pi
       for sigma in (0.2, 0.4):  #Sigma with values 
            for gamma in (0.5, 0.8):   #Gamma values            
              gabor_label = 'Gabor' + str(num)  #Label Gabor columns as Gabor1, Gabor2, etc.
              ksize=9
              kernel = cv.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, psi, ktype=cv.CV_32F)    
              kernels.append(kernel)
              #Now filter the image and add values to a new column 
              fimg = cv.filter2D(img2, cv.CV_8UC3, kernel) # convolution
              filtered_img = fimg.reshape(-1)
              df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
              #print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
              num += 1  #Increment for gabor column label               
   return df

In [ ]:
## Add the patient ID and Label then save Gabor features 
def get_Gabor(path_fold):
  features = pd.DataFrame()

  for path in tqdm(path_fold):
    # save the gabor filters
    feature_df = Gabor(path)
    ## Take mean, varience, skenwness, kurtosis and minimum for each filter
    df1=feature_df.mean().to_frame().T
    df2=feature_df.var().to_frame().T
    df3=feature_df.skew().to_frame().T
    df4=feature_df.kurtosis(axis=0).to_frame().T
    df5=feature_df.min().to_frame().T

    frames = [df1, df2, df3, df4, df5]
    result = pd.concat(frames, axis=1, join='inner')

    # add patient ids
    result.insert(0, "0", path.rsplit('/', 1)[1].split('-', 3)[2])
    # add end point
    result["label"]=int(path.find('_M_')!=-1)

    features = pd.concat([features, result])

  return features

## Saving files as csv

In [ ]:
####################################################
## Returns the csv file for a given folder, train test, magnifications, and feature name (LBP)
def get_file_fv(paths,feature_name,fold, train_test, magnification, get_feature_type):
  mag_dictionary = {0:'40',1:'100',2:'200',3:'400'}
  fv = get_feature_type(paths[fold,train_test,magnification])
  train_df=pd.DataFrame(np.row_stack(fv))
  train_df.to_csv(f'/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/fold{fold+1}/{tt_dict[train_test]}/{feature_name}_f{fold+1}_{tt_dict[train_test]}_{mag_dictionary[magnification]}x_fv.csv', sep='\t')

# Extraction

In [ ]:
#Load images paths
with open('/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/paths.p', 'rb') as handle:
    paths = pickle.load(handle)

In [ ]:
#Extraction of LBP pattern for all folds, train-test and magnifications
for fold in range(5):
   for tt in range(2):
     for mag in range(4):
       get_file_fv(paths=paths, feature_name = 'lbp', fold=fold, train_test=tt, magnification=mag, get_feature_type=get_LBP)

In [ ]:
#Extraction of PFTAS pattern for all folds, train-test and magnifications
for fold in range(5):
   for tt in range(2):
     for mag in range(4):
       get_file_fv(paths=paths, feature_name = 'PFTAS', fold=fold, train_test=tt, magnification=mag, get_feature_type=get_PFTAS)

In [ ]:
#Extraction of Gabor filters
for fold in range(5):
   for tt in range(2):
     for mag in range(4):
       get_file_fv(paths=paths, feature_name = 'Gabor', fold=fold, train_test=tt, magnification=mag , get_feature_type=get_Gabor)

In [ ]:
#GLCM extraction
#Extractor settings and nanme of features to extract
features_names = ['JointEnergy', 'Contrast','Correlation', 'SumSquares','Idm','SumAverage', 'ClusterTendency','SumEntropy','JointEntropy','DifferenceVariance','DifferenceEntropy','Imc1','Imc2'] #Name of features to extract
extractor, num_features = extractor_setting(features_names) #Return extractor and number of features

for fold in range(5):
  for tt in range(2):
    for mag in range(4):
      GLCM_for_folder(paths=paths,f=fold, tt=tt, mag=mag, num_features=num_features, extractor=extractor) #Create and save feature matrices